https://planetachatbot.com/tutorial-chatbot-usando-nltk-keras/
https://github.com/hiteshmishra708/python-chatbot

In [18]:
!pip install tensorflow 
!pip install keras 
!pip install pickle 
!pip install nltk


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [84]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import json
import pickle

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
#Added David
import re
#END Added

lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Definición del personaje del chatbot para el entrenamiento
personaje = 'UNGAUNGA' 
# se remplaza el nombre definido por en la cadena <personaje>
#Convert to string and replace
obj_str = json.dumps(intents).replace('<personaje>', personaje)

#Get obj back with replacement
intents = json.loads(obj_str)
# se guardan los intents personalizados
with open('intents_personalizado.json', 'w') as outfile:
    json.dump(intents, outfile)
print(intents)

# intents: gruppi di conversazioni-tipo
# patterns: possibili interazioni dell'usuario
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenizzo ogni parola
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # aggiungo all'array documents
        documents.append((w, intent['tag']))
        print(w,intent['tag'])
        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

# preparazione per l'addestramento della rete
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # bag of words
    bag = []
    # lista di tokens
    pattern_words = doc[0]
    # lemmatizzazione dei token
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # se la parola matcha, inserisco 1, altriment 0
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

training = np.array(training)
# creazione dei set di train e di test: X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

# creazione del modello
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))#128
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))#64
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)#epochs =300
model.save('chatbot_model.h5', hist)

print("model created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
{'intents': [{'tag': 'saludos', 'patterns': ['Hola ', '¿Hola, que tal estas?', '¿Qué tal estas?', '¡Que pasa!', '¡Hola!', '¡Hey!', '¡Buenos días!'], 'responses': ['Hola que tal, ¿En que te puedo ayudar?', '¡Hola!, ¿En qué te puedo ayudar?'], 'context': ['nombre']}, {'tag': 'despedida', 'patterns': ['Adios', 'Nos vemos', 'Ciao', 'Que pases un buen día, adios', 'Hasta pronto', 'Me piro', 'Me voy', 'salir'], 'responses': ['Adios <name>, que tengas un buen día', 'Hasta luego <name>', 'Ciao <name>!'], 'context': ['']}, {'tag': 'agradecimiento', 'patterns': ['Gracias', 'Muchas gracias', 'Gracias a ti', 'Fué de gran ayuda', 'Increible, gracias', 'Gracias por ayudarme'], 'responses': ['¡Feliz de ayudar!', 'Ha sido un placer ayudarte'], 'context': ['']}, {'tag': 'nombre', 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


10/10 [==============================] - 0s 2ms/step - loss: 2.1320 - accuracy: 0.1087
Epoch 2/300
10/10 [==============================] - 0s 2ms/step - loss: 2.0564 - accuracy: 0.1739
Epoch 3/300
10/10 [==============================] - 0s 2ms/step - loss: 1.9359 - accuracy: 0.2826
Epoch 4/300
10/10 [==============================] - 0s 2ms/step - loss: 1.8276 - accuracy: 0.2826
Epoch 5/300
10/10 [==============================] - 0s 2ms/step - loss: 1.6908 - accuracy: 0.4130
Epoch 6/300
10/10 [==============================] - 0s 2ms/step - loss: 1.7358 - accuracy: 0.2609
Epoch 7/300
10/10 [==============================] - 0s 2ms/step - loss: 1.5606 - accuracy: 0.4348
Epoch 8/300
10/10 [==============================] - 0s 3ms/step - loss: 1.6713 - accuracy: 0.3478
Epoch 9/300
10/10 [==============================] - 0s 2ms/step - loss: 1.6002 - accuracy: 0.3478
Epoch 10/300
10/10 [==============================] - 0s 2ms/step - loss: 1.3729 - accuracy: 0.4783
Epoch 11/300
10/10 [=

In [94]:
import nltk, json, random, pickle
#Added David
from nltk import word_tokenize,pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
#END Added
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents_personalizado.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# preprocessamento input user
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# creazione bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def calcula_prediccion(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    #ERROR_THRESHOLD = 0.25
    ERROR_THRESHOLD = 0
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        print("intent:", classes[r[0]], " probability:", str(r[1]))
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getRespuesta(ints, intents_json, entrada, name):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    # Control de intents de ayuda nombre del personaje al principio
    if(re.match(rf"^{re.escape(personaje)}\b(?!\w)",str(entrada))):
      tag = 'ayuda'
    elif(tag == 'ayuda'):
      del ints[0]
      tag = ints[0]['intent']
    for i in list_of_intents:
        if(i['tag'] == tag):
          # Added David
          # Control del diálogo
          #get_DialogFlow(dialogFlow, tag)
          if (tag == 'minombre'):
            # Se extraen los NER
            tokens = word_tokenize(entrada)
            tag = pos_tag(tokens)
            ne_tree = nltk.ne_chunk(tag)
            print(ne_tree)
            # Extract PERSON tag for name
            if name == '':
              #name = re.findall('PERSON(.\S+)\/NNP', str(ne_tree))
              if (re.findall('\(S .+?\(PERSON(.\S+)\/NNP', str(ne_tree))):
                name = re.findall('\(S .+?\(PERSON(.\S+)\/NNP', str(ne_tree))
              print(name)
            result = random.choice(i['responses'])
            if name :
              result = re.sub('<name>', str(name[0]).strip(), result)
            else:
              result = re.sub('<name>', '', result)
            print(result)
            break   
            # END Added
          result = random.choice(i['responses'])
          if name :
              result = re.sub('<name>', str(name[0]).strip(), result)
          else:
              result = re.sub('<name>', '', result)
          break
    return result, name

def inizia(msg, name):
    ints = calcula_prediccion(msg, model)
    res , name = getRespuesta(ints, intents, msg, name)
    return res, name

def update_DialogFlow(dialogFlow, ints, res, action):
    dialogFlow.append[{'estado:'}]
    return True
def get_DialogFlow(dialogFlow, ints):
    return True

# Control del diálogo
dialogFlow = [{'estado':'inicio','intent':'','respuesta':'','intent_previo':''}]
# Introducción del reto
introduccion ='La vida de '+personaje+' es muy dura, y necesita tu ayuda para sobrevivir, ¿le puedes ayudar?'
usuario = ''
print('Para salir, escribe "salir"')
name = ''
print('¡Hola!, me llamo ', personaje,'!!')
print(introduccion)
intro=True
while usuario != 'salir':
  usuario = str(input(""))
  if (intro):
    ints = calcula_prediccion(usuario, model)
    if ints[0]['intent']=='afirmativo':
      res , name = getRespuesta(ints, intents, usuario, name)
      #dialogFlow[] 
    else:
      res = 'Vaya!, bueno nos vemos en otra ocasión.'
      usuario = 'salir'
    intro=False
  else:
    res, name  = inizia(usuario, name)
  print(personaje,':' + res)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
Para salir, escribe "salir"
¡Hola!, me llamo  UNGAUNGA !!
La vida de UNGAUNGA es muy dura, y necesita tu ayuda para sobrevivir, ¿le puedes ayudar?
si
intent: afirmativo  probability: 0.99899346
intent: norespuesta  probability: 0.0006976419
intent: minombre  probability: 0.00027709565
intent: saludos  probability: 1.4774172e-05
intent: nombre  probability: 1.4541041e-05
intent: despedida  probability: 2.1591616e-06
intent: ayuda  probability: 2.2090758e-07
intent: agradecimiento  probability: 1.4983091e-07
UNGAUNGA :¡Muchas gracias!
Hola
intent

https://www.youtube.com/watch?v=1lwddP0KUEg

https://www.machinecurve.com/index.php/2021/03/16/easy-chatbot-with-dialogpt-machine-learning-and-huggingface-transformers/